## Data preparation

### Libraries and zip file

In [1]:
import pandas as pd
import numpy as np
import zipfile
import matplotlib.pyplot as plt


In [2]:
    
# import zip file with csv 
ou_zip = zipfile.ZipFile('../data/anonymisedData.zip') 

# save separate csvs
registrations = pd.read_csv(ou_zip.open('studentRegistration.csv'))
courses = pd.read_csv(ou_zip.open('courses.csv'))
students = pd.read_csv(ou_zip.open('studentInfo.csv'))
student_vle = pd.read_csv(ou_zip.open('studentVle.csv'))
vle = pd.read_csv(ou_zip.open('vle.csv'))
student_assessments = pd.read_csv(ou_zip.open('studentAssessment.csv'))
assessments = pd.read_csv(ou_zip.open('assessments.csv'))

### course information
* merge course information from course table - merge on code_module and code_presentation



In [15]:
# merge students and courses
student_regist = pd.merge(students, courses, on=['code_module', 'code_presentation'], validate='many_to_one')

#final.count()

### registrations

* merge registration table with student table
* drop rows with missing 'imd_band' and 'date_registration'


NOTE: some students have more than one registration, but not on the same module_presentation (code_module, code_presentation)


In [16]:
# merge registrations
student_regist = pd.merge(student_regist, registrations, on=['code_module', 'code_presentation', 'id_student'], how = 'left', validate='1:1')
#final.count()

In [17]:
# drop missing value rows (date_registration, imd_band)
student_regist.dropna(subset=['date_registration', 'imd_band'], inplace=True)
#final.count()

### prediction point

* one main goal of predicting outcome is to understand whether 'intervention' is needed - thus early identification and action is valuable.
* therefore predictions need to be made as early as possible, or at variable points in time.
* it would not make sense to make an early preediction using information that is not avaialable at that point in time - so the data needs to be subset based on a prediction point.
* `prediction_point` is the point in time from the start of the course (in days) at which the prediction - it removes:
  * students who have already unregistered - their outcome is already known
  * information that is not available at that point in time (assessments, vle)

In [29]:
# prediction point = days from start of course
prediction_point = 200

# prediction point must be less than course length, integer, and greater than 0
if not isinstance(prediction_point, int) or prediction_point <= 0 or prediction_point >= max(student_regist['module_presentation_length']):
    print("Error: Invalid prediction point. \n\nPlease provide an integer value greater than 0 and less than the maximum course length. \n\nThis is the number of days from the start of the course for which you want to predict the outcome.")
else:
    # withdrawn or failed before prediction point - remove
    withdrawn_fail_condition = (student_regist['final_result'].isin(['Withdrawn', 'Fail'])) & (student_regist['date_unregistration'] <= prediction_point)
    student_regist.loc[withdrawn_fail_condition, 'status'] = 'remove_outcome_known'
    # if unregister after prediction point - keep
    unregister_after_condition = student_regist['date_unregistration'] > prediction_point
    student_regist.loc[unregister_after_condition, 'status'] = 'keep'
    # if no unregistration date - keep
    no_unregistration_condition = student_regist['date_unregistration'].isna()
    student_regist.loc[no_unregistration_condition, 'status'] = 'keep'
    # default case
    student_regist.loc[~(withdrawn_fail_condition | unregister_after_condition | no_unregistration_condition), 'status'] = 'query'

query_rows = student_regist[student_regist['status'] == 'query'] | student_regist[student_regist['status'].isna()]


if not query_rows.empty:
    print("The following rows need investigation.  They are excluded from the following analysis: \n")
    print(query_rows)
    final = student_regist[~student_regist.isin(query_rows)].dropna()

    



date_unregistration blanks need to be populated with the end date of the course - these students are still registered

In [30]:
# replace missing date_unreg with module_presentation_length
student_regist['date_unregistration'] = student_regist['date_unregistration'].fillna(student_regist['module_presentation_length'])
#final.count()

### dataset for the model - adding engagement data



In [38]:
# remove rows from final student df which are not needed
model_final = student_regist[student_regist['status'] != 'remove_outcome_known']
model_final.count()
#model_final.head(20)

code_module                   22149
code_presentation             22149
id_student                    22149
gender                        22149
region                        22149
highest_education             22149
imd_band                      22149
age_band                      22149
num_of_prev_attempts          22149
studied_credits               22149
disability                    22149
final_result                  22149
module_presentation_length    22149
date_registration             22149
date_unregistration           22149
status                        22149
dtype: int64

### assessments in time - add to table

prepare assessments data

In [42]:
# merge 'assessments' and 'courses' on 'code_module' and 'code_presentation'
course_assess = assessments.merge(courses[['code_module', 'code_presentation', 'module_presentation_length']], on=['code_module', 'code_presentation'], how='left')

# fill in the missing 'date' values with course final week (as per literature)
value_to_fill = course_assess['module_presentation_length'] - 3
course_assess['date'] = course_assess['date'].fillna(value_to_fill)

In [43]:
# merge student_assessments with course_assess
stu_assess = pd.merge(student_assessments, course_assess, on=['id_assessment'], how='left')
stu_assess.count()


id_assessment                 173912
id_student                    173912
date_submitted                173912
is_banked                     173912
score                         173739
code_module                   173912
code_presentation             173912
assessment_type               173912
date                          173912
weight                        173912
module_presentation_length    173912
dtype: int64

In [37]:
# drop students who have no score for an assessment

# rows with a missing scoare
missing_score_rows = stu_assess[stu_assess['score'].isna()]

# student ids with missing score
unique_ids_missing = missing_score_rows['id_student'].unique()

# remove students with missing score from student df
model_final = model_final[~model_final['id_student'].isin(unique_ids_missing)]
#model_final.count()



code_module                   22023
code_presentation             22023
id_student                    22023
gender                        22023
region                        22023
highest_education             22023
imd_band                      22023
age_band                      22023
num_of_prev_attempts          22023
studied_credits               22023
disability                    22023
final_result                  22023
module_presentation_length    22023
date_registration             22023
date_unregistration           22023
status                        22023
dtype: int64

In [44]:
# drop rows with missing score
stu_assess.dropna(subset=['score'], inplace=True)
#stu_assess.count()

id_assessment                 173739
id_student                    173739
date_submitted                173739
is_banked                     173739
score                         173739
code_module                   173739
code_presentation             173739
assessment_type               173739
date                          173739
weight                        173739
module_presentation_length    173739
dtype: int64

fixing date issues - see other file for details

In [45]:
# removing students with negative or extreme date_submitted values
# removing rows from merged_assess

# get max module length
max_module_length = stu_assess['module_presentation_length'].max()

# get students with negative or extreme date_submitted values
greater_than_max_length = stu_assess[stu_assess['date_submitted'] > max_module_length]['id_student'].unique()
less_than_zero = stu_assess[stu_assess['date_submitted'] < 0]['id_student'].unique()

# remove students from model_final
students_to_remove = set(greater_than_max_length) | set(less_than_zero)
model_final = model_final[~model_final['id_student'].isin(students_to_remove)]

# remove rows from merged_assess
merged_assess = stu_assess[~stu_assess['id_student'].isin(students_to_remove)]



removing extreme submission dates

In [49]:
# removing students with negative or extreme date_submitted values
# removing rows from merged_assess

# get max module length
max_module_length = stu_assess['module_presentation_length'].max()

# get students with negative or extreme date_submitted values
greater_than_max_length = stu_assess[stu_assess['date_submitted'] > max_module_length]['id_student'].unique()
less_than_zero = stu_assess[stu_assess['date_submitted'] < 0]['id_student'].unique()

# remove students from model_final
students_to_remove = set(greater_than_max_length) | set(less_than_zero)
model_final = model_final[~model_final['id_student'].isin(students_to_remove)]

# remove rows from merged_assess
stu_assess = stu_assess[~stu_assess['id_student'].isin(students_to_remove)]



reduce stu_ssess to `prediction_point` value

In [ ]:
# reduce data by prediction point
model_student_assess = stu_assess[stu_assess['date'] <= prediction_point]



create new assessment features - suitable for all modules

In [51]:
# get expected assessment details

model_course_assess = course_assess[course_assess['date'] <= prediction_point]
#model_course_assess.count()

# expected assessment count per module_presentation
expected_ass = model_course_assess.groupby(['code_module', 'code_presentation'])['id_assessment'].count().reset_index()
expected_ass = expected_ass.rename(columns={'id_assessment': 'exp_sub_count'})

# excpeted assessment date sum
date_sum = model_course_assess.groupby(['code_module', 'code_presentation'])['date'].sum().reset_index()
expected_ass = expected_ass.merge(date_sum, on=['code_module', 'code_presentation'], how='left')
expected_ass = expected_ass.rename(columns={'date': 'exp_sub_date_sum'})

#expected_ass.count()

code_module          22
code_presentation    22
exp_sub_count        22
exp_sub_date_sum     22
dtype: int64

In [52]:
# summarise students assessments by module_presentation
student_assessment_summary = model_student_assess.groupby(['id_student', 'code_module', 'code_presentation']).agg(
    count_id_assessment=('id_assessment', 'count'),
    sum_score=('score', 'sum'),
    sum_date=('date', 'sum')
).reset_index()
#student_assessment_summary.count()

In [55]:
# merge student_assessment_summary and expected_ass on code_module and code_presentation
merged_assess_summary = student_assessment_summary.merge(expected_ass, on=['code_module', 'code_presentation'])

# calculate the new features
merged_assess_summary['prop_submissions'] = merged_assess_summary['count_id_assessment'] / merged_assess_summary['exp_sub_count']
merged_assess_summary['avg_score'] = merged_assess_summary['sum_score'] / merged_assess_summary['exp_sub_count']
merged_assess_summary['submission_distance'] = merged_assess_summary['exp_sub_date_sum'] - merged_assess_summary['sum_date']


### VLE data

steps:


* merge vle with courses - maybe?
* 
* merge vle with student_vle
* assess missing values - these still need to be kept, probably as 0
* assess vle against dates
* reduce vle to `prediction_point` value
* create new features - tbd
* summary_vle df
* check for duplicates - group on stu_id, mod_presenation
* 

this merge may not be needed - only if I use course_length

In [66]:
# merge 'vle' and 'courses' on 'code_module' and 'code_presentation'
course_vle = vle.merge(courses, on=['code_module', 'code_presentation'], how='left')

#missing_values = course_vle.isnull().sum()
#print("Missing values:\n", missing_values, "\n")

# drop week_from and week_to columns
course_vle = course_vle.drop(['week_from', 'week_to'], axis=1)

#missing_values = course_vle.isnull().sum()
#print("Missing values:\n", missing_values, "\n")

In [64]:
# number of vle types per module_presentation - potential feature

group_vle = vle.groupby(['code_module', 'code_presentation'])['activity_type'].nunique().reset_index()
group_vle.rename(columns={'activity_type': 'type_count'}, inplace=True)

# prints
print(group_vle)


   code_module code_presentation  type_count
0          AAA             2013J           9
1          AAA             2014J           9
2          BBB             2013B          10
3          BBB             2013J          10
4          BBB             2014B          10
5          BBB             2014J          10
6          CCC             2014B           9
7          CCC             2014J           9
8          DDD             2013B          11
9          DDD             2013J          10
10         DDD             2014B          10
11         DDD             2014J          10
12         EEE             2013J          11
13         EEE             2014B          11
14         EEE             2014J          11
15         FFF             2013B          14
16         FFF             2013J          16
17         FFF             2014B          15
18         FFF             2014J          16
19         GGG             2013J           7
20         GGG             2014B           7
21        

this merge may not be needed.

In [73]:
#merge vle with student_vle
all_stu_vle = pd.merge(student_vle, course_vle, on=['id_site', 'code_module', 'code_presentation'], how='left')



269

In [72]:
# Filter the rows where 'date' is greater than 'module_presentation_length'
filtered_data = all_stu_vle[all_stu_vle['date'] > all_stu_vle['module_presentation_length']]

# Print the resulting DataFrame
print(filtered_data)


Empty DataFrame
Columns: [code_module, code_presentation, id_student, id_site, date, sum_click, activity_type, module_presentation_length]
Index: []


In [71]:


# aggregations for each column
aggregations = {
    'id_site': 'count',
    'activity_type': 'nunique',
    'sum_click': 'sum',
    'date': lambda x: x.nunique()
}

# group and apply the aggregations
grouped_data = all_stu_vle.groupby(['code_module', 'code_presentation', 'id_student']).agg(aggregations).reset_index()

# rename the columns
grouped_data.rename(columns={
    'id_site': 'stu_activity_count',
    'activity_type': 'stu_activity_type_count',
    'sum_click': 'stu_total_clicks',
    'date': 'stu_days_active'
}, inplace=True)

# print
print(grouped_data.head())



  code_module code_presentation  id_student  stu_activity_count  \
0         AAA             2013J       11391                 196   
1         AAA             2013J       28400                 430   
2         AAA             2013J       30268                  76   
3         AAA             2013J       31604                 663   
4         AAA             2013J       32885                 352   

   stu_activity_type_count  stu_total_clicks  stu_days_active  
0                        6               934               40  
1                        7              1435               80  
2                        6               281               12  
3                        8              2158              123  
4                        7              1034               70  


In [62]:
all_stu_vle.groupby(['id_student', 'code_module', 'code_presentation']).count().reset_index()

,id_student,code_module,code_presentation,id_site,date,sum_click,activity_type,module_presentation_length
0,6516,AAA,2014J,662,662,662,662,662
1,8462,DDD,2013J,300,300,300,300,300
2,8462,DDD,2014J,4,4,4,4,4
3,11391,AAA,2013J,196,196,196,196,196
4,23629,BBB,2013B,59,59,59,59,59
...,...,...,...,...,...,...,...,...
29223,2698257,AAA,2013J,267,267,267,267,267
29224,2698535,CCC,2014B,190,190,190,190,190
29225,2698535,EEE,2013J,709,709,709,709,709
29226,2698577,BBB,2014J,237,237,237,237,237


TODO

vle data


Merge:
merged_assess_summary with final student

Final table:
* create new features for 'intake', 'year' and 'subject' from code_module and code_presentation
* rename cols
* reorder cols
* check and drop nulls
* create new id and drop separate ids